In [50]:
# def filter_tweet(tweet):
# # Remove any tweets that don't match our criteria.
#     if not tweet_matches_criteria(tweet):
#         return
# # Process the remaining tweets.
# process_tweet(tweet)
# def process_tweet(tweet):
# # Annotate the tweet dictionary with any other information we need.
#     tweet["sentiment"] = get_sentiment(tweet)
# # Store the tweet.
# store_tweet(tweet)
# def store_tweet(tweet):
# # Saves a tweet for later processing. ...

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import configparser
import time
import tweepy

In [67]:
path_conf = 'conf/settings.ini'
config = configparser.ConfigParser()
config.read(path_conf)
keys = config['KEYS']

In [68]:
TWITTER_APP_KEY = keys['TWITTER_APP_KEY']
TWITTER_APP_SECRET = keys['TWITTER_APP_SECRET']
TWITTER_KEY = keys['TWITTER_KEY']
TWITTER_SECRET = keys['TWITTER_SECRET']

auth = tweepy.OAuthHandler(TWITTER_APP_KEY, TWITTER_APP_SECRET)
auth.set_access_token(TWITTER_KEY, TWITTER_SECRET)

api = tweepy.API(auth)

In [69]:
# build listener for new tweets

# create a listener that prints the text of any tweet that comes from the Twitter API.
class StreamListener(tweepy.StreamListener):
  def __init__(self, time_limit=60):
    self.start_time = time.time()
    self.limit = time_limit
    self.saveFile = open('twitter.csv', 'w')
    super(StreamListener, self).__init__()
  
  def on_status(self, status):
    if (time.time() - self.start_time) < self.limit:
      if hasattr(status, "retweeted_status"):
        return
      else:
        if hasattr(status, "extended_tweet"):
          text = status.extended_tweet["full_text"]
        else:
          text = status.text
        # save text to file
        self.saveFile.write(text)
        self.saveFile.write('\n')
        print(text)
        # return tweet text
        #print(status['extended_tweet']['full_text'])
    else:
      self.saveFile.close()
      return False

  def on_error(self, status_code):
    if status_code == 420:
      return False

In [70]:
stream_listener = StreamListener(time_limit=10)

In [74]:
stream_listener.running = False

```
description = status.user.description
loc = status.user.location
text = status.text
coords = status.coordinates
name = status.user.screen_name
user_created = status.user.created_at
followers = status.user.followers_count
id_str = status.id_str
created = status.created_at
retweets = status.retweet_count
bg_color = status.user.profile_background_color
```

interesting objects attributes
```
text: short text with link
user: object(id, name, screen_name, location)
extended_tweet(full_text, user_mentions)
```

In [72]:
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
ltrack = ["trump", "clinton", "hillary clinton", "donald trump"]
stream.filter(track=ltrack, is_async=True)

Resign and walk himself directly to jail a s confess everything he has done
@hoob_mark @vo5tok You’re an actual ret*rd if you think trump had enough votes in the senate to do anything. I’m blocking you for being a bitch
@atrupar Did fox cover tRump wearing a mask?
#democrats and #USA WAKE UP!

You are letting communism take over the US in the form of liberalism and anti racism.

What’s going on has zero to do w racism &amp; everything to do w #liberal agendas and easing them through. 

#Trump needs to be re-elected or we will all be killed off
@troymation @JDoss84 @Myish1 @722marine @wglady @TheRightMelissa @RealCandaceO Ya Boy Keith...

You may want to do some more research on @RealCandaceO. Girl’s got rent to pay and black conservatives trashing Dems pays EXTREMELY well. 

Trump has been a con artist all his life...con artists support other con artists.

🩸60 yrs ago👇🏽
https://t.co/cmSoNuMPGN https://t.co/cQ9JJemx7p
@Saiiyko @NYMami777 @MaElviraSalazar @KookieNyo Like I said I am not 